In [1]:
# Import block

import alpha_shape_Mrestani as ash  # From Mrestani repo
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Utility functions

def um_to_px(coords, pxum=6.25):
    # pxum is pixels per micron
    return coords*pxum - 0.5

In [635]:
# Load Nikon data file with points

dbscan_file = '20220526_L1_NMJ1_A2R67_DBSCAN_v2.xlsx'

#Read Excel file, extract columns of interest (**change out sheet name (B1_Ib_DBSCAN) for desired sheet**)
data = pd.read_excel(dbscan_file, sheet_name= 'B1_Ib_DBSCAN', usecols= [1, 2, 5, 37])

#Remove lateral localization accuracy points below 50nm (0.05 microns)
data_loc_acc = data.loc[data["Lateral Localization Accuracy"] <= 0.05]

#Sort data by Cluster ID
data_sorted = data_loc_acc.sort_values(by = ['Clusters - ID'], ascending = True )
data_sorted

,X Position [µm],Y Position [µm],Lateral Localization Accuracy,Clusters - ID
123,18.412582,11.790802,0.046826,2
162,18.440820,11.762816,0.029610,2
26,18.460686,11.766873,0.041973,2
28,18.410106,11.803893,0.022242,2
278,18.443359,11.799879,0.027378,2
...,...,...,...,...
661,17.547169,11.901045,0.040318,284
292,17.607776,11.957288,0.024288,284
673,17.641285,11.922378,0.038294,284
282,17.564518,11.935361,0.033214,284


In [636]:
# Load max projection image from raw nd2 video wih Fiji/ImageJ to maintain alignment
max_proj_file = 'MAX_20220526_CacHalo_JF646_L1_A2R67_000.nd2 - C=0.png'

Im = plt.imread(max_proj_file)

In [637]:
# Plot max projection image and all non-noise poits on top

# pop out to allow zoom, etc
%matplotlib qt  

pts_all = um_to_px( data[data['Clusters - ID']>=0].iloc[:,0:3].to_numpy() ) # pixels
pts_all_ids = data[data['Clusters - ID']>=0].iloc[:,3].to_numpy()

plt.figure(figsize=(12,12))
plt.imshow(Im.mean(axis=2), cmap = "gray")  # Plot greyscale
plt.scatter(pts_all[:,0],pts_all[:,1],5,pts_all_ids)
plt.axis('image')
plt.show()

In [638]:
# restore inline plotting
%matplotlib inline  

In [639]:
cluster_ids = data_sorted.iloc[:,3].unique()  # Check ID column
cluster_ids = cluster_ids[cluster_ids>=0]  # Reject noise cluster of -1 Column ID

alpha_param = 500

cluster_stats = np.full( (len(cluster_ids),5), np.nan )
cluster_shapes = []
for kk, cid in enumerate(cluster_ids):
    print(f'Working on cluster {cid}')
    if cid>=0:
        coords = data[data['Clusters - ID']==cid].iloc[:,0:2].to_numpy()
        num_coords = len(coords)
        cluster_stats[kk,0] = cid  # Identifier
        cluster_stats[kk,1] = ash.get_alpha_area(coords, alpha_param)  # Area, um^2
        cluster_stats[kk,2] = 2*np.sqrt(cluster_stats[kk,1]/np.pi)
        cluster_stats[kk,3] = num_coords / cluster_stats[kk,1]  # Density, num/um^2
        cluster_stats[kk,4] = num_coords   # number of locs in cluster
       #Average the lateral loc accuracy column for each cluster ID?
        cluster_shapes.append( ash.get_alpha_shape(coords, alpha_param) ) # Full shape object

Working on cluster 2
Working on cluster 55
Working on cluster 125
Working on cluster 146
Working on cluster 211
Working on cluster 279
Working on cluster 284


In [ ]:
# TODO: Plot alpha shapes on top of points
        #Can we add in average value from localization accuracy column for each cluster?

In [640]:
cluster_stats_pd = pd.DataFrame(cluster_stats, columns = ['Cluster_ID','ClusterArea ( $\mu m^2$ )','ClusterDiameter ( $\mu m$ )','Density','# of Locs'])
cluster_stats_pd

,Cluster_ID,ClusterArea ( $\mu m^2$ ),ClusterDiameter ( $\mu m$ ),Density,# of Locs
0,2.0,0.006208,0.088903,6282.633735,39.0
1,55.0,0.019044,0.155717,8559.036646,163.0
2,125.0,0.013871,0.132895,7137.160623,99.0
3,146.0,0.016102,0.143183,10806.259560,174.0
4,211.0,0.011594,0.121500,4139.985355,48.0
5,279.0,0.022182,0.168058,12757.882655,283.0
6,284.0,0.017024,0.147228,9045.919143,154.0
